**1-4**
See codes for answers.

**5**
The accuracy, confusion matrix, and F-1 score are reported in In[17].

**6**
The accuracy, confusion matrix, and F-1 score are reported in In[21].

**7**
I used train_data_stop_removed to generate training tokens. I fed into the MLPClassifier model the list of embedded sentences outputted by BERT. The accuracy, confusion matrix, and F-1 score are reported in In[29].

**8**
*a)* 
Fine-tuning in the context of this paper refers to the process of adjusting a pre-trained model using a smaller dataset that is specific to the task at hand. In the case of the paper you provided, the authors fine-tune models on the ED dataset to improve their performance in generating empathetic responses in a dialogue setting. We could use such fine-tuning on building an empathetic chatbot by pretraining data and focus on tuning the model to face task-specific data.

*b)* 
Transformer's ability to capture contextuak dependencies allows the application model to weight different parts of the input sequence differently, which is crucial for understanding the context in a conversation, especially when generating empathetic responses that depend on the emotional tone of the dialogue.

*c)* 
P@1,100: precision retrieving the correct test candidate out of 100 test candidates.
AVG BLEU: average of BLEU-1,-2,-3,-4. Used for evaluating the quality of machine-generated text by comparing it to a reference text.
PPL: perplexity, a measure of how well a probability distribution predicts a sample.

*d)*
P@1,100 would fit the most for reflecting the system's ability to provide a relevant and human-like response among a diverse set of options.

*e)*
The paper demonstrates that fine-tuning models on the newly introduced ED dataset leads to improvements in both automated metrics and human evaluations. Besides, the use of BERT-based models shows improvements in performance, indicating the potential benefits of leveraging contextualized pretraining. 

In [1]:
import os
import re

import pandas as pd
import numpy as np

import gensim
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
import nltk
nltk.download('punkt')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


from itertools import compress
import collections

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JkL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JkL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# first step is to preprocess the data

# Importing the datasets
train = pd.read_csv('train.csv', error_bad_lines=False)
valid = pd.read_csv('valid.csv', error_bad_lines=False)
test = pd.read_csv('test.csv', error_bad_lines=False)

C:\Users\JkL\AppData\Local\Temp\ipykernel_42740\198559996.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train = pd.read_csv('train.csv', error_bad_lines=False)
b'Skipping line 2355: expected 8 fields, saw 10\nSkipping line 36628: expected 8 fields, saw 12\nSkipping line 49433: expected 8 fields, saw 10\nSkipping line 56957: expected 8 fields, saw 10\nSkipping line 65019: expected 8 fields, saw 10\n'
C:\Users\JkL\AppData\Local\Temp\ipykernel_42740\198559996.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  valid = pd.read_csv('valid.csv', error_bad_lines=False)
b'Skipping line 3: expected 8 fields, saw 9\nSkipping line 5: expected 8 fields, saw 9\nSkipping line 8: expected 8 fields, saw 9\nSkipping line 10: expected 8 fields, saw 9\nSkipping line 12: expected 8 fields, saw 9\nSkipping line 14

b'Skipping line 3: expected 8 fields, saw 9\nSkipping line 5: expected 8 fields, saw 9\nSkipping line 8: expected 8 fields, saw 9\nSkipping line 10: expected 8 fields, saw 9\nSkipping line 12: expected 8 fields, saw 9\nSkipping line 14: expected 8 fields, saw 9\nSkipping line 16: expected 8 fields, saw 9\nSkipping line 18: expected 8 fields, saw 9\nSkipping line 20: expected 8 fields, saw 9\nSkipping line 22: expected 8 fields, saw 9\nSkipping line 25: expected 8 fields, saw 9\nSkipping line 27: expected 8 fields, saw 9\nSkipping line 30: expected 8 fields, saw 9\nSkipping line 32: expected 8 fields, saw 9\nSkipping line 35: expected 8 fields, saw 9\nSkipping line 37: expected 8 fields, saw 9\nSkipping line 39: expected 8 fields, saw 9\nSkipping line 41: expected 8 fields, saw 9\nSkipping line 43: expected 8 fields, saw 9\nSkipping line 45: expected 8 fields, saw 9\nSkipping line 48: expected 8 fields, saw 9\nSkipping line 50: expected 8 fields, saw 9\nSkipping line 53: expected 8 fiel

In [3]:
# select "utterance" and "context" as your X and y
# only select {'sad', 'jealous', 'joyful', 'terrified'} categories
# use pandas.loc function to do it!


#X_train = train.loc[train['context'].isin({'sad', 'jealous', 'joyful', 'terrified'})][['utterance', 'context']]

#X_test =train.loc[train['context'].isin({'sad', 'jealous', 'joyful', 'terrified'})][['utterance', 'context']]

X_train = train.loc[train['context'].isin({'sad', 'jealous', 'joyful', 'terrified'})][['utterance', 'context']]

X_test =test.loc[test['context'].isin({'sad', 'jealous', 'joyful', 'terrified'})][['utterance', 'context']]

In [4]:
# Getting the train labels; this will be used for SGD classifier
train_labels_unique = list(X_train['context'].unique())
label_mapper = {}
num = 0
for label in train_labels_unique:
    label_mapper[label] = num
    num += 1


train_labels = list(X_train['context'])
train_labels_encoded = []
for label in train_labels:
    train_labels_encoded.append(label_mapper[label])
    
# Getting test labels
labels_test = list(X_test['context'])
labels_encoded_test = []
for label in labels_test:
    labels_encoded_test.append(label_mapper[label])
labels_encoded_test = np.array(labels_encoded_test)

# note train and test labels are in train_labels_encoded and labels_encoded_test

In [5]:
# data preprocessing, remove punctuations from the sentence
# DO NOT REMOVE STOPWORDS here
# follow this article should be good to go 
# https://medium.com/@arunm8489/getting-started-with-natural-language-processing-6e593e349675
# https://machinelearningmastery.com/clean-text-machine-learning-python/
import string

def remove_punctuation(text):
    return ''.join([char for char in text if char not in (string.punctuation)])
train_data_list_cleaned = X_train['utterance'].apply(remove_punctuation).tolist()
test_data_list_cleaned = X_test['utterance'].apply(remove_punctuation).tolist()

In [6]:
### 2
## Converting the utterances into a sparse bag-of-words 

# use sklearn should be good
train_count_vectorizer = CountVectorizer()
X = train_count_vectorizer.fit_transform(train_data_list_cleaned)
encoding = X.toarray()

In [7]:
# Converting counts to binary resukt
for arr in encoding:
    arr[arr > 0] = 1
    
# now you have the sparse encoding

In [8]:
### 3 
## The shortcomings with the previous representation are ?

# so we should possibly remove stop words.
# Getting the list of stopwords and appending additional words to it
stopwords_list = list(set(stopwords.words('english')))
stopwords_list.extend(['comma', ''])  
stopwords_list.remove("not")
print("a" in stopwords_list)

True


In [9]:
# remove the tokens in the stopwords list from utterance

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords_list]
    return ' '.join(filtered_words)
def remove_numbers(text):
    pattern = r'[0-9]'
    new_string = re.sub(pattern, '', text)
    return new_string.replace('comma', '')



train_data_stop_removed = X_train['utterance'].apply(remove_stopwords).apply(remove_punctuation).apply(remove_numbers).tolist()
test_data_stop_removed = X_test['utterance'].apply(remove_stopwords).apply(remove_punctuation).apply(remove_numbers).tolist()

In [10]:
test_data_stop_removed

['Im sad ive read article newborn girl died parents believe medication doctors',
 'born premature home hard time breathing instead taking doctor parents praying',
 'yes believe God prayers goodness gracious please take children hospital let God heal doctors',
 'im excited im finally going visit parents next month see  years',
 'moved US stayed homecountry',
 'Im scared bugs found one hair yesterday almost died',
 'Ugh hate bugs passion creepy',
 'One morning mother came house pick daughter school told saw daughters cat road hit tell daughter loss beloved pet',
 'around  took hard went ahead let stay home school knew well day',
 'One saddest things people underestimate andor capable of',
 'Thats perfectly natural sound like kind person though quickly regained bearings Disappointments like usually come good lessons',
 'met old flame recently go expected',
 'Well right gave big hug introduced new boyfriend',
 'know right warning anything turned nice guy thought little misleading part',
 '

In [11]:
# Creating the bag of words encoding again  
train_count_vectorizer = CountVectorizer()
X_train = train_count_vectorizer.fit_transform(train_data_stop_removed)

train_one_hot_encoding = X_train.toarray()

for arr in train_one_hot_encoding:
    arr[arr > 0] = 1

In [12]:
### 4. Normalization
# Normalizing the training data using tfidf transformer 
# let us use sklearn again

train_tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
train_embedding_tfidf_transformer = train_tfidf_transformer.fit_transform(train_one_hot_encoding)

In [13]:
### 5. Building an SGD Classifier
X_train = train_embedding_tfidf_transformer
y_train = np.array(train_labels_encoded)

In [14]:
clf = SGDClassifier()
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html
clf.fit(X_train, y_train)

# you can evaluate the training accuracy by predicted result and true label y_train

SGDClassifier()

In [15]:
# Using training data vocabulary on test data so that the features are consistent    
test_count_vectorizer = CountVectorizer(vocabulary = train_count_vectorizer.get_feature_names())
X_test = test_count_vectorizer.fit_transform(test_data_stop_removed)

test_one_hot_encoding = X_test.toarray()

E:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
# Normalizing the test data  
test_tfidf_transformer = TfidfTransformer(smooth_idf=False,use_idf=True)
test_embedding_tfidf_transformer = test_tfidf_transformer.fit_transform(test_one_hot_encoding)

# Getting predictions on test data
test_predicted_labels = clf.predict(test_embedding_tfidf_transformer)

E:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:1632: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [17]:
# do some evaluation on the test set
print('Test accuracy :', np.mean(labels_encoded_test == test_predicted_labels))

f1_score_vector = f1_score(labels_encoded_test, test_predicted_labels, average=None)

print('F1 score :', np.mean(labels_encoded_test == test_predicted_labels))

print('Confusion matrix :', confusion_matrix(labels_encoded_test, test_predicted_labels))

print('f1 score using SGD classifier is :', np.mean(f1_score_vector))

Test accuracy : 0.6430555555555556
F1 score : 0.6430555555555556
Confusion matrix : [[117   7  16  15]
 [ 18 115  37  17]
 [ 24  24 125  22]
 [ 16  33  28 106]]
f1 score using SGD classifier is : 0.6447558519738392


In [18]:
### 6. Classifier using pretrained embeddings

# Tokenizing the data
train_tokens = [nltk.word_tokenize(sentences) for sentences in train_data_stop_removed]
train_y = np.array(train_labels_encoded)

test_tokens = [nltk.word_tokenize(sentences) for sentences in test_data_stop_removed]
test_y = np.array(labels_encoded_test)

In [19]:
# Loading the pretrained word2vec model from Google
# download the model here: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# follow this article to do it
# https://towardsdatascience.com/using-word2vec-to-analyze-news-headlines-and-predict-article-success-cdeda5f14751

In [20]:
def get_word_embeddings(tokens, model):
    embeddings = []
    for sentence in tokens:
        valid_words = [word for word in sentence if word in model]
        if valid_words:
            sentence_embedding = np.mean([model[word] for word in valid_words], axis=0)
            embeddings.append(sentence_embedding)
        else:
            embeddings.append(np.zeros(model.vector_size))
    return np.array(embeddings)

In [21]:
train_X = get_word_embeddings(train_tokens, model)
test_X = get_word_embeddings(test_tokens, model)

In [22]:
classifier = MLPClassifier(hidden_layer_sizes=(256,), max_iter=200, random_state=95891)
classifier.fit(train_X, train_y)

E:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(256,), random_state=95891)

In [23]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
predictions = classifier.predict(test_X)
conf_matrix = confusion_matrix(test_y, predictions)
f1 = f1_score(test_y, predictions, average='weighted')
accuracy = accuracy_score(test_y, predictions)

print("Confusion Matrix:\n", conf_matrix)
print("F1 Score:", f1)
print("Test Accuracy:", accuracy)

Confusion Matrix:
 [[123  12  11   9]
 [ 33 104  28  22]
 [ 25  28 113  29]
 [ 12  35  20 116]]
F1 Score: 0.6308135720420535
Test Accuracy: 0.6333333333333333


In [24]:
### 7. Classifier using pretrained BERT

from transformers import DistilBertTokenizer, DistilBertModel
# load the tokenizer and the model of distilbert-base-uncased
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# tokenize the text, then input the tokens (and masks) into the model to get the output

# use the BERT output to train a MLP classifier


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
import torch
input_ids = []
for sentence in train_data_stop_removed:
    input_ids.append(tokenizer(sentence, return_tensors='pt'))

Token indices sequence length is longer than the specified maximum sequence length for this model (1140 > 512). Running this sequence through the model will result in indexing errors


In [26]:
import torch

# Define the batch size for tokenization
batch_size = 1000  # You can adjust this based on your available memory

# Tokenize and obtain BERT embeddings for the training data
train_sentence_embeddings = []

for i in range(0, len(train_data_stop_removed), batch_size):
    batch = train_data_stop_removed[i:i + batch_size]
    encoding = tokenizer(batch, truncation=True, padding='max_length', return_tensors='pt', max_length=188)
    
    with torch.no_grad():
        outputs = model(input_ids=encoding['input_ids'], attention_mask=encoding['attention_mask'])
    
    sentence_embeddings = outputs.last_hidden_state[:, 0, :]
    train_sentence_embeddings.append(sentence_embeddings)

    print(f"current: {i} of {len(train_data_stop_removed)}", end=" ")

train_sentence_embeddings = torch.cat(train_sentence_embeddings)


current: 0 of 9796 current: 1000 of 9796 current: 2000 of 9796 current: 3000 of 9796 current: 4000 of 9796 current: 5000 of 9796 current: 6000 of 9796 current: 7000 of 9796 current: 8000 of 9796 current: 9000 of 9796 

In [27]:
test_sentence_embeddings = []
for i in range(0, len(test_data_stop_removed), batch_size):
    batch = test_data_stop_removed[i:i + batch_size]
    encoding = tokenizer(batch, truncation=True, padding='max_length', return_tensors='pt', max_length=188)
    
    with torch.no_grad():
        outputs = model(input_ids=encoding['input_ids'], attention_mask=encoding['attention_mask'])
    
    sentence_embeddings = outputs.last_hidden_state[:, 0, :]
    test_sentence_embeddings.append(sentence_embeddings)
    print(f"current: {i} of {len(test_data_stop_removed)}", end=" ")
test_sentence_embeddings = torch.cat(test_sentence_embeddings)

current: 0 of 720 

In [28]:
mlp_classifier = MLPClassifier(hidden_layer_sizes=(256,), max_iter=200, random_state=95891)
mlp_classifier.fit(train_sentence_embeddings, train_labels)

MLPClassifier(hidden_layer_sizes=(256,), random_state=95891)

In [29]:
X_test =test.loc[test['context'].isin({'sad', 'jealous', 'joyful', 'terrified'})][['utterance', 'context']]
test_labels = list(X_test['context'])
# Assuming you want to evaluate the model on the same data
# This is not typical, but if it's your requirement:
y_pred_test = mlp_classifier.predict(test_sentence_embeddings)
f1_test = f1_score(test_labels, y_pred_test, average='weighted')
accuracy_test = accuracy_score(test_labels, y_pred_test)
cm_test = confusion_matrix(test_labels, y_pred_test)

# Print results for the training set (not a typical evaluation)
print(f"Training F1 Score: {f1_test}")
print(f"Training Accuracy: {accuracy_test}")
print("Training Confusion Matrix:")
print(cm_test)

Training F1 Score: 0.5907086330456974
Training Accuracy: 0.5972222222222222
Training Confusion Matrix:
[[ 76  45  39  23]
 [ 15 103  50  19]
 [ 15  22 135  23]
 [  7  14  18 116]]
